In [10]:
#Version del scraper de Maldita 2.0
#Esta versión ha sido desarrollada para de una forma limpia poder scrapear maldita seleccionando un página en concreto.

In [11]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import pickle
import unicodedata

In [12]:
pagina = 20

In [13]:
def get_urls(link, lista_enlaces):
    
    page = requests.get(link)
    
    soup2 = BeautifulSoup(page.content, 'html.parser')

    todo2 = soup2.find('div', class_='post-cards-list')
    urls = todo2.find_all('a')

    enlaces = []
    for url in urls:
        #print(url.attrs['href'])
        enlaces.append(url.attrs['href'])
    for e in enlaces[1::2]:
        lista_enlaces.append(e)
    lista_enlaces

In [14]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="content-title pl-3 pr-3 mb-3 mt-0").get_text()
    #print(title)

    date = soup3.find("h6",class_="pl-3 pr-3 mt-3 text-uppercase").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="article-body")
    body2 = body.find_all("p", class_="")

    for e in body2:
        content_list.append(unicodedata.normalize("NFKD", e.get_text()))
    
    content = ''.join(content_list)
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url, "Fuente": "Maldita.es"}
    
    return article

In [15]:
lista_enlaces = []
suficiente = True

#Cargo las URLS.
print('Cargando URLs...')

link =  "https://maldita.es/malditobulo/page/" + str(pagina) + "/"

arrayurls= get_urls(link, lista_enlaces)
        
lista_articulos = []   

for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)
    
print(len(lista_enlaces))
print(lista_enlaces)

Cargando URLs...
24
['https://maldita.es/malditobulo/2020/04/07/presentadora-informativos-telecinco-angeles-blanco-gracias-a-dios/', 'https://maldita.es/malditobulo/2020/04/07/mujer-pancarta-machismo-mata-mas-que-coronavirus-periodista-de-lasexta/', 'https://maldita.es/malditobulo/2020/04/07/limon-bicarbonato-coronavirus-te/', 'https://maldita.es/malditobulo/2020/04/07/leire-pajin-asesoramiento-gobierno-covid-19/', 'https://maldita.es/malditobulo/2020/04/07/listado-productos-justifican-supermercado-guardia-civil-no-guia-actuacion-oficial/', 'https://maldita.es/malditobulo/2020/04/07/20000-mascarillas-inmovilizadas-aduana-zaragoza/', 'https://maldita.es/malditobulo/2020/04/07/bulo-control-ciudadania-geolocalizacion-moviles/', 'https://maldita.es/malditobulo/2020/04/06/mercadona-cupon-500-euros-coronavirus/', 'https://maldita.es/malditobulo/2020/04/06/proposicion-ley-modificacion-injurias-corona-ultrajes/', 'https://maldita.es/malditobulo/2020/04/06/ataudes-vacios-expuestos-venta-nave-ba

In [16]:
lista_articulos

[{'Title': 'No, la presentadora de informativos Telecinco Ángeles Blanco no dice "en Alemania gracias a Dios se complica la situación": dice "gracias Darío"',
  'Publication_date': '07/04/2020',
  'Content': 'Se están difundiendo por Twitter publicaciones que aseguran que la presentadora de los informativos de fin de semana de Telecinco, Ángeles Blanco, dice que "en Alemania gracias a Dios se complica la situación" por el coronavirus. Algunas de estas publicaciones acumulan hasta 19.000 me gusta y más de 5.000 compartidos. Es un bulo: la presentadora no dice "gracias a Dios" sino "gracias Darío", en referencia al corresponsal que habla previamente. En los vídeos que se están difundiendo sólo se ve esa intervención de la presentadora y no se ve lo que sucede antes. Sin embargo, en el vídeo completo de los informativos del mediodía del 4 de abril se puede ver que antes de este corte, habla un corresponsal en Roma llamado Darío Menor. Cuando este acaba su intervención, la pre

In [17]:
#Guardamos los datos:
from datetime import date

date = date.today().strftime("%d-%m-%Y")

filename = 'Maldita_FN_Pag-' + str(pagina) + '_'+ date
print(filename)

outfile = open(filename,'wb')
pickle.dump(lista_articulos, outfile)  
    
outfile.close()

Maldita_FN_Pag-20_20-06-2020


In [18]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': 'No, la presentadora de informativos Telecinco Ángeles Blanco no dice "en Alemania gracias a Dios se complica la situación": dice "gracias Darío"',
  'Publication_date': '07/04/2020',
  'Content': 'Se están difundiendo por Twitter publicaciones que aseguran que la presentadora de los informativos de fin de semana de Telecinco, Ángeles Blanco, dice que "en Alemania gracias a Dios se complica la situación" por el coronavirus. Algunas de estas publicaciones acumulan hasta 19.000 me gusta y más de 5.000 compartidos. Es un bulo: la presentadora no dice "gracias a Dios" sino "gracias Darío", en referencia al corresponsal que habla previamente. En los vídeos que se están difundiendo sólo se ve esa intervención de la presentadora y no se ve lo que sucede antes. Sin embargo, en el vídeo completo de los informativos del mediodía del 4 de abril se puede ver que antes de este corte, habla un corresponsal en Roma llamado Darío Menor. Cuando este acaba su intervención, la pre